In [106]:
from collections import deque
import math
import random

import attr
import gym
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
from pytorch_monitor import init_experiment, monitor_module
# from smooth import smooth  # timeseries smoothing function
import torch
from torch import nn
import torch.nn.functional as F


cartpole = gym.make('CartPole-v1')
lunarlander = gym.make('LunarLander-v2')
plt.style.use('seaborn-white')

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'  

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


# TODO: add device spec for every tensor

In [107]:
@attr.s
class Memory(deque):
    """ Experience Replay Memory class. """
    size = attr.ib()
    minibatch_size = attr.ib()

    def append(self, thing):
        if len(self) > self.size - 1:
            self.popleft()
        return super().append(thing)

    def sample(self):
        batch_size = min(len(self), self.minibatch_size)
        data = random.sample(self, batch_size)
        states = torch.stack([record[0] for record in data])
        actions = torch.tensor([record[1] for record in data], dtype=torch.long)
        rewards = torch.tensor([record[2] for record in data], dtype=torch.float)
        states_ = torch.stack([record[3] for record in data])
        dones = torch.tensor([record[4] for record in data], dtype=torch.long)
        return (states, actions, rewards, states_, dones)


class ValueDistribution(torch.nn.Module):
    def __init__(self, state_shape, action_shape, vmin, vmax, num_atoms=51, num_hidden1_units=64, num_hidden2_units=64):
        super().__init__()
        self.state_shape = state_shape
        self.action_shape = action_shape
        self.vmin = vmin
        self.vmax = vmax
        self.num_atoms = num_atoms
        self.atoms = torch.linspace(self.vmin, self.vmax, self.num_atoms)
        self.linear1 = nn.Linear(self.state_shape, num_hidden1_units)
        self.linear2 = nn.Linear(num_hidden1_units, num_hidden2_units)
        self.linear3 = nn.Linear(num_hidden2_units, num_hidden2_units)
        self.out = nn.Linear(num_hidden2_units, self.action_shape * self.num_atoms)
        
    def forward(self, x):
        """ Return (actions x atoms). """
        x1 = F.relu(self.linear1(x))
        x2 = F.relu(self.linear2(x1))
        x3 = self.linear3(x2)
        x4 = self.out(x3).reshape(-1, self.action_shape, self.num_atoms)
        out = F.softmax(x4, dim=2)  # (actions x atoms)
        if x.dim() == 1:
            batch_size = 1
        else:
            batch_size = x.size(0)
        assert out.size() == torch.Size((batch_size, self.action_shape, self.num_atoms))
        if hasattr(self, 'monitor'):
            self.monitor('x1', x1, track_data=True, track_grad=True)
            self.monitor('x2', x2, track_data=True, track_grad=True)
            self.monitor('x3', x3, track_data=True, track_grad=True)
            self.monitor('x4', x4, track_data=True, track_grad=True)
            self.monitor('out', out, track_data=True, track_grad=True)
        return out
    
    def predict_action_values(self, states):
        """ Return (batch-size x actions). """
        distribution = self.forward(states)
        weighted_distribution = distribution * self.atoms
        out = weighted_distribution.sum(dim=2).squeeze()  # (batch-size x actions)
        dims = states.dim()
        assert out.size() == torch.Size((self.action_shape,))
        return out
        
    def get_action(self, state):
        """ Given a single state, return the action. """
        values = self.predict_action_values(state)
        action = values.argmax()
        return action


In [125]:
def categorical_loss(online_net, target_net, transitions, discount): 
    states, actions, rewards, states_, dones = transitions
    not_dones = (1 - dones).type(torch.FloatTensor)
    atoms = target_net.atoms
    probabilities = target_net.forward(states_)
    Q_x_ = (probabilities * atoms).sum(2)
    batch_size = states.shape[0]
    assert Q_x_.shape == torch.Size((batch_size, target_net.action_shape)), f'Got: {Q_x_.shape}, expected: {(batch_size, target_net.action_shape)}'
    a_star = Q_x_.argmax(dim=1) 
    assert a_star.shape == torch.Size((batch_size,)), f'Got {a_star.shape}, expected: ((batch_size,))'
    
    # compute the projected probability:
    delta_z = (target_net.vmax - target_net.vmin)/(target_net.num_atoms - 1)    
    # select only the probabilities distributions for the a_star actions:
    probabilities = probabilities[range(batch_size), a_star]
    T_zj = rewards.unsqueeze(1) + discount * atoms * not_dones.unsqueeze(1)
    b_j = (T_zj.clamp(target_net.vmin, target_net.vmax) - target_net.vmin) / delta_z  # correct
    lo = b_j.floor()
    hi = b_j.ceil()
    m = torch.zeros(batch_size, target_net.num_atoms, dtype=torch.float)
    row_idxs = torch.range(0, batch_size-1, dtype=torch.long).unsqueeze(1).expand_as(m)
    m[row_idxs, lo.type(torch.long)] += (probabilities * (hi - b_j))
    m[row_idxs, hi.type(torch.long)] += (probabilities * (b_j - lo))

    # x enthropy is Sigma <true> log <unnatural>, so for us is: target log(online)
    online_distribution = online_net.forward(states)[range(batch_size), actions]
    return -( m * online_distribution.log() ).sum(1).mean()

In [119]:
@attr.s
class CategoricalAgent:
    env = attr.ib()
    discount = attr.ib(default=0.99)
    epsilon_max = attr.ib(default=1.0)
    epsilon_min = attr.ib(default=0.01)
    annealing_const = attr.ib(default=.001)  # aka Lambda
    minibatch_size = attr.ib(default=32)
    memory_size = attr.ib(default=int(1e6))
    num_episodes = attr.ib(default=1000)  # num of episodes in a training epoch
    render_every = attr.ib(default=20)  # set to zero to turn off rendering
    update_target_every = attr.ib(default=200)
    vmin = attr.ib(default=-10)
    vmax = attr.ib(default=10)
    num_atoms = attr.ib(default=51)
    learning_rate = attr.ib(default=0.000001)
    monitor_every = attr.ib(default=50)
    logger = attr.ib(default=None)
    
    def __attrs_post_init__(self):
        self.steps = 0
        state_shape = self.env.observation_space.shape[0]
        self.memory = Memory(self.memory_size, self.minibatch_size)
        self.action_shape = self.env.action_space.n
        self.online_net = ValueDistribution(state_shape=state_shape, action_shape=self.action_shape, vmin=self.vmin, vmax=self.vmax, num_atoms=self.num_atoms)
        self.target_net = ValueDistribution(state_shape=state_shape, action_shape=self.action_shape, vmin=self.vmin, vmax=self.vmax, num_atoms=self.num_atoms)
        self.target_net.load_state_dict(self.online_net.state_dict())
        self.optimizer = torch.optim.Adam(self.online_net.parameters(), lr=self.learning_rate)
        self.reset_data_recorders()        

    def reset_data_recorders(self):
        self.episode_rewards = []
        self.episode_losses = []
        self.td_errors = []
        self.online_net_q_values = []
        self.target_net_q_values = []
        self.w1_gradient = []
        self.w2_gradient = []
        self.steps = 0

    def render(self, episode):
        if self.render_every and episode % self.render_every == 0:
            self.env.render()

    def training_progress_report(self, episode):
        last_ep = self.episode_rewards[-1]
        ten_ep_mean = sum(self.episode_rewards[-10:])/len(self.episode_rewards[-10:])
        hundred_ep_mean = sum(self.episode_rewards[-100:])/len(self.episode_rewards[-100:])
        return f'Ep: {episode} // steps: {self.steps} // last ep reward: {last_ep:.2f} // {min(10, len(self.episode_rewards[-10:]))}-ep mean: {ten_ep_mean:.2f} // {min(100, len(self.episode_rewards[-100:]))}-ep mean: {hundred_ep_mean:.2f}'

    def replay(self):
        batch = self.memory.sample()
        loss = categorical_loss(self.online_net, self.target_net, batch, self.discount)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()/self.minibatch_size

    def monitor(self):
        if self.steps % self.monitor_every == 0:
            self.target_net.monitoring(True)
        else:
            self.target_net.monitoring(False)

    def train(self):
        for episode in range(self.num_episodes):
            episode_done = False
            episode_reward = 0
            episode_loss = 0
            state = torch.tensor(self.env.reset(), dtype=torch.float)
            self.target_net_q_values.append(self.target_net.predict_action_values(state).max().item())
            if self.steps == 0:
                self.logger.add_graph(self.target_net, state)
            writer.add_scalar('Target net Q values', self.target_net_q_values[-1], self.steps)                
            while not episode_done:
                epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * math.exp(-self.annealing_const * self.steps)
                self.steps += 1                
                if random.random() < epsilon:
                    action = random.randint(0, self.action_shape-1)
                else:
                    action = self.online_net.get_action(state).item()
                self.render(episode)
                self.monitor()
                state_, reward, episode_done, _ = self.env.step(action)
                state_ = torch.tensor(state_, dtype=torch.float)
                episode_reward += reward
                self.memory.append((state, action, reward, state_, episode_done))
                episode_loss += self.replay()
                state = state_
                if self.steps % self.update_target_every == 0:
                    self.target_net.load_state_dict(self.online_net.state_dict())
                if episode_done:
                    self.episode_rewards.append(episode_reward)
                    print(self.training_progress_report(episode), end='\r', flush=True)
                    writer.add_scalar('train loss', episode_loss, self.steps)
                    writer.add_scalar('episode reward', episode_reward, self.steps)                    
        self.env.close()

    def test(self):
        pass

In [124]:
monitor_config = {
    'title':'Test Monitor',
    'log_dir':'categorical-dqn',
    'random_seed': 0
}
writer, monitor_config = init_experiment(monitor_config)
monitor_config
agent = CategoricalAgent(
    cartpole, 
    learning_rate=0.000000001, 
    logger=writer,
    monitor_every=10,
    num_episodes=400,
    update_target_every=100,    
)
monitor_module(
    agent.target_net, writer, 
    track_data=True,
    track_grad=True,
    track_update=True,
    track_update_ratio=True
)
agent.train()

{'log_dir': 'categorical-dqn',
 'random_seed': 0,
 'run_dir': 'categorical-dqn/Jul-17-18@19:35:36-DaydreamNation.local',
 'run_name': 'Jul-17-18@19:35:36-DaydreamNation.local',
 'tag': 'Experiment Config: Test Monitor :: Jul-17-18@19:35:36\n',
 'title': 'Test Monitor'}

/Users/liavkoren/Envs/ai-gym/lib/python3.6/site-packages/torch/onnx/utils.py:365: UserWarning: ONNX export failed on ATen operator reshape because torch.onnx.symbolic.reshape does not exist
  .format(op_name, op_name))


[23] > <ipython-input-116-0d960ce73e16>(30)categorical_loss()
-> return loss
(Pdb++) l
 25  	    # x enthropy is Sigma <true> log <unnatural>, so for us is: target log(online)
 26  	    online_distribution = online_net.forward(states)[range(batch_size), actions]
 27  	    # return -(m * online_distribution.log()).sum(1).mean()
 28  	    loss = -(m * online_distribution.log())
 29  	    import pdb; pdb.set_trace()
 30  ->	    return loss
[EOF]
(Pdb++) loss.backward()
*** RuntimeError: grad can be implicitly created only for scalar outputs
(Pdb++) ones = torch.ones_like(loss)
(Pdb++) loss.backward(ones)
(Pdb++) target_net.parameters()
<generator object Module.parameters at 0x1234706d0>
(Pdb++) target_net.linear1
Linear(in_features=4, out_features=64, bias=True)
(Pdb++) dir(target_net.linear1)
['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__',

RuntimeError: grad can be implicitly created only for scalar outputs

In [172]:
arm0 = 2.7
arm1 = 5
arm2 = -3.4


class Bandit:
    @staticmethod
    def step(action):
        if action == 0:
            return torch.tensor([0.]), np.random.normal(arm0, 1)
        elif action == 1:
            return torch.tensor([1.]), np.random.normal(arm1, 1)
        elif action == 2:
            return torch.tensor([2.]), np.random.normal(arm2, 3)            
        else:
            raise ValueError('Invalid action', action)
            
    
@attr.s
class BanditAgent:
    num_arms = attr.ib(default=3)
    num_rounds = attr.ib(default=2000)
    discount = attr.ib(default=0.99)
    epsilon_max = attr.ib(default=1.0)
    epsilon_min = attr.ib(default=0.01)
    annealing_const = attr.ib(default=.001)  # aka Lambda
    minibatch_size = attr.ib(default=32)
    memory_size = attr.ib(default=int(1e6))
    vmin = attr.ib(default=-10)
    vmax = attr.ib(default=10)
    num_atoms = attr.ib(default=51)
    learning_rate = attr.ib(default=0.00025)
    monitor_every = attr.ib(default=5)
    logger = attr.ib(default=None)
    update_target_every = attr.ib(default=100)
    xavier = attr.ib(default=False)

    def __attrs_post_init__(self):
        self.online_net = ValueDistribution(state_shape=1, action_shape=self.num_arms, vmin=-10, vmax=10, num_atoms=self.num_atoms, num_hidden1_units=40, num_hidden2_units=40,)
        self.target_net = ValueDistribution(state_shape=1, action_shape=self.num_arms, vmin=-10, vmax=10, num_atoms=self.num_atoms, num_hidden1_units=40, num_hidden2_units=40,)        
        if self.xavier:
            for param in bandit_agent.online_net.parameters():
                if param.dim() < 2:
                    continue
                nn.init.xavier_normal_(param)        
        self.target_net.load_state_dict(self.online_net.state_dict())
        self.optimizer = torch.optim.Adam(self.online_net.parameters(), lr=self.learning_rate)
        self.reset_data_recorders()
        self.memory = Memory(1e6, 32)
        
    def reset_data_recorders(self):
        self.episode_rewards = []
        self.episode_losses = []
        self.q_values = []
        self.steps = 0

    def monitor(self):
        if self.steps % 20 == 0:
            print(f'{self.steps}... ', end='\r', flush=True)

        if self.steps % self.monitor_every == 0:
            self.target_net.monitoring(True)
        else:
            self.target_net.monitoring(False)
            
            
    def replay(self):
        batch = self.memory.sample()
        loss = categorical_loss(self.online_net, self.target_net, batch, self.discount)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()/self.minibatch_size
            
    def train(self):
        for episode in range(self.num_rounds):
            state = torch.tensor([1], dtype=torch.float)
            self.q_values.append(self.target_net.predict_action_values(state).max().item())
            if self.steps == 0:
                self.logger.add_graph(self.target_net, state)
            writer.add_scalar('Target net Q values', self.q_values[-1], self.steps)                

            epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * math.exp(-self.annealing_const * self.steps)
            self.steps += 1                
            if random.random() < epsilon:
                action = random.randint(0, self.num_arms-1)
            else:
                action = self.online_net.get_action(state).item()
            self.monitor()
            state_, reward = Bandit.step(action)
            state_ = torch.tensor(state_, dtype=torch.float)
            episode_reward = reward
            self.memory.append((state, action, reward, state_, torch.tensor([0])))
            episode_loss = self.replay()
            state = state_
            writer.add_scalar('train loss', episode_loss, self.steps)
            writer.add_scalar('episode reward', episode_reward, self.steps)   
            
            if self.steps % self.update_target_every == 0:
                self.target_net.load_state_dict(self.online_net.state_dict())
            

    def test(self):
        pass       


In [183]:


monitor_config = {
    'title':'Bandit test',
    'log_dir':'categorical-dqn',
    'random_seed': 0
}
writer, monitor_config = init_experiment(monitor_config)
bandit_agent = BanditAgent(
    logger=writer, 
    learning_rate=0.00000001, 
    num_rounds=10000, 
    monitor_every=50,
    xavier=False,
)
monitor_config
monitor_module(
    bandit_agent.target_net, writer, 
    track_data=True,
    track_grad=True,
    track_update=True,
    track_update_ratio=True
)
bandit_agent.train()

{'log_dir': 'categorical-dqn',
 'random_seed': 0,
 'run_dir': 'categorical-dqn/Jul-18-18@12:03:36-DaydreamNation.local',
 'run_name': 'Jul-18-18@12:03:36-DaydreamNation.local',
 'tag': 'Experiment Config: Bandit test :: Jul-18-18@12:03:36\n',
 'title': 'Bandit test'}

/Users/liavkoren/Envs/ai-gym/lib/python3.6/site-packages/torch/onnx/utils.py:365: UserWarning: ONNX export failed on ATen operator reshape because torch.onnx.symbolic.reshape does not exist
  .format(op_name, op_name))


In [184]:
arm0
arm1
arm2

bandit_agent.target_net.predict_action_values(torch.tensor([0], dtype=torch.float))
bandit_agent.target_net.predict_action_values(torch.tensor([1], dtype=torch.float))
bandit_agent.target_net.predict_action_values(torch.tensor([2], dtype=torch.float))

2.7

5

-3.4

tensor(1.00000e-02 *
       [ 3.7750, -2.3753, -2.0731])

tensor(1.00000e-02 *
       [-2.7775, -8.6544, -6.7508])

tensor([-0.0754, -0.1647, -0.1222])

In [159]:
bandit_agent.episode_losses

[]

In [182]:
bandit_agent.target_net.linear1.weight
bandit_agent.target_net.linear1.weight.grad.norm()

Parameter containing:
tensor([[-0.0154],
        [ 0.5342],
        [-0.8230],
        [-0.7359],
        [-0.3852],
        [ 0.2682],
        [-0.0291],
        [ 0.7950],
        [-0.0887],
        [ 0.2546],
        [-0.2919],
        [-0.1930],
        [-0.9553],
        [-0.6623],
        [-0.4020],
        [ 0.0370],
        [ 0.4038],
        [ 0.6000],
        [-0.6779],
        [-0.4355],
        [ 0.3542],
        [ 0.8314],
        [-0.2058],
        [ 0.7550],
        [-0.1603],
        [ 0.1058],
        [ 0.9166],
        [-0.9277],
        [-0.6295],
        [-0.2418],
        [-0.3898],
        [ 0.8670],
        [-0.6482],
        [-0.4505],
        [-0.7076],
        [-0.9366],
        [-0.5837],
        [ 0.8596],
        [ 0.4520],
        [ 0.4847]])

tensor(25.1479)